##### Data to Use

***

[Contract Awards in Investment Project Financing](https://dev.socrata.com/foundry/finances.worldbank.org/kdui-wcs3)

> It contains the countries that which the project was awarded a fund, the sector in which the project will be performed, and the amount in USD, together with awarding date.

- API: https://finances.worldbank.org/resource/kdui-wcs3.json

***

[Contract Awards in Investment Project Financing (since FY 2001 - FY 2016)](https://dev.socrata.com/foundry/finances.worldbank.org/4bhp-2q7b)

> It contains the contract awards which precedes the above dataset, containing data about preceding years.

- API: https://finances.worldbank.org/resource/4bhp-2q7b.json


In [1]:
# import relevant libraries
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pycountry

##### Importing and Preprocessing the Data

In [19]:
#
# IMPORT THE DATA
# 

csv_new = "Contract_Awards_in_Investment_Project_Financing.csv"
csv_old = "Contract_Awards_in_Investment_Project_Financing__since_FY_2001_-_FY_2016_.csv"

df_new = pd.read_csv(csv_new)
df_old = pd.read_csv(csv_old)

unique_countries = set()

for each in df_old["Borrower Country"].unique():
    unique_countries.add(each)

for each in df_new["Borrower Country"].unique():
    unique_countries.add(each)

#
# CLEAN THE DATA
#

# drop the rows with "Borrower Country" as NaN in the new dataset
df_new.dropna(subset=["Borrower Country"], inplace=True)

# change "Turkey" to "Turkiye" in the old dataset
df_old.loc[df_old["Borrower Country"] == "Turkey", "Borrower Country"] = "Turkiye"

#
# MERGE THE DATAFRAMES
# 

# merge the dataframes with the relevant categories as following:
# Fiscal Year, Region, Borrower Country, Project Global Practice, Supplier Contract Amount (USD)
df_new = df_new[['Fiscal Year',
                 'Borrower Country',
                 'Supplier Contract Amount (USD)']]
df_old = df_old[['Fiscal Year',
                 'Borrower Country',
                 'Supplier Contract Amount']]

# rename the columns to match the old dataframe
df_new.rename(columns={'Supplier Contract Amount (USD)': 'Supplier Contract Amount'}, inplace=True)

# merge the dataframes, removing any duplicates
df = pd.concat([df_new, df_old], ignore_index=True)

df.rename(columns={'Fiscal Year': 'Year', 'Borrower Country': 'Country', 'Supplier Contract Amount': 'Award Amount'}, inplace=True)

df = df.groupby(['Year', 'Country']).sum().reset_index()

df.head()

,Year,Country,Award Amount
0,2001,Africa,2259197.0
1,2001,Albania,31015512.0
2,2001,Algeria,53339668.0
3,2001,Angola,3976897.0
4,2001,Aral Sea,2922224.0


In [20]:
# considering the "Country" column, create a new column of "Country Code"
df["Country Code"] = df["Country"].apply(lambda x: pycountry.countries.get(name=x).alpha_3 if pycountry.countries.get(name=x) else None)

In [21]:
df.head()

,Year,Country,Award Amount,Country Code
0,2001,Africa,2259197.0,None
1,2001,Albania,31015512.0,ALB
2,2001,Algeria,53339668.0,DZA
3,2001,Angola,3976897.0,AGO
4,2001,Aral Sea,2922224.0,None


In [22]:
# print the "Country" values correspond to a None "Country Code"
df[df["Country Code"].isnull()]["Country"].unique()

array(['Africa', 'Aral Sea', 'Bolivia', 'Central America', 'Central Asia',
       "Cote d'Ivoire", 'Eastern Africa', 'Egypt, Arab Republic of',
       'Gambia, The', 'Kosovo', 'Kyrgyz Republic', 'Moldova',
       'OECS Countries', 'Red Sea and Gulf of Aden', 'Slovak Republic',
       'St. Kitts and Nevis', 'St. Lucia', 'Tanzania', 'Turkiye',
       'Venezuela, Republica Bolivariana de', 'Vietnam',
       'West Bank and Gaza', 'Western Africa', 'Western Balkans',
       'Yemen, Republic of', 'Congo, Democratic Republic of',
       'Congo, Republic of', 'Mekong', 'St. Vincent and the Grenadines',
       'Caribbean', 'Latin America', 'Southern Africa', 'Czech Republic',
       'Europe and Central Asia', 'East Asia and Pacific', 'Caucasus',
       'World', 'Andean Countries', 'Central Africa',
       'Middle East and North Africa', 'Pacific Islands', 'Asia',
       'South Asia', 'Eastern and Southern Africa', 'St Maarten',
       'Western and Central Africa', 'Multi-Regional', 'South East 

In [23]:
custom_country_mapping = {
    'Africa': None, 'Aral Sea': None, 'Bolivia': 'BOL', 'Central America': None, 'Central Asia': None,
    "Cote d'Ivoire": 'CIV', 'Eastern Africa': None, 'Egypt, Arab Republic of': 'EGY',
    'Gambia, The': 'GMB', 'Kosovo': None, 'Kyrgyz Republic': 'KGZ', 'Moldova': 'MDA',
    'OECS Countries': None, 'Red Sea and Gulf of Aden': None, 'Slovak Republic': 'SVK',
    'St. Kitts and Nevis': 'KNA', 'St. Lucia': 'LCA', 'Tanzania': 'TZA', 'Turkiye': 'TUR',
    'Venezuela, Republica Bolivariana de': 'VEN', 'Vietnam': 'VNM',
    'West Bank and Gaza': None, 'Western Africa': None, 'Western Balkans': None,
    'Yemen, Republic of': 'YEM', 'Congo, Democratic Republic of': 'COD',
    'Congo, Republic of': 'COG', 'Mekong': None, 'St. Vincent and the Grenadines': 'VCT',
    'Caribbean': None, 'Latin America': None, 'Southern Africa': None, 'Czech Republic': 'CZE',
    'Europe and Central Asia': None, 'East Asia and Pacific': None, 'Caucasus': None,
    'World': None, 'Andean Countries': None, 'Central Africa': None,
    'Middle East and North Africa': None, 'Pacific Islands': None, 'Asia': None,
    'South Asia': None, 'Eastern and Southern Africa': None, 'St Maarten': None,
    'Western and Central Africa': None, 'Multi-Regional': None, 'South East Asia': None
}

In [26]:
custom_country_mapping["Bolivia"]

'BOL'

In [30]:
# update the "Country Code" column with the custom mapping
df["Country Code"] = df["Country"].apply(
    lambda x: custom_country_mapping[x] 
    if x in custom_country_mapping.keys()
    else (
        pycountry.countries.get(name=x).alpha_3 
        if pycountry.countries.get(name=x)
        else None
        )
    )

In [28]:
df.head()

,Year,Country,Award Amount,Country Code
0,2001,Africa,2259197.0,None
1,2001,Albania,31015512.0,ALB
2,2001,Algeria,53339668.0,DZA
3,2001,Angola,3976897.0,AGO
4,2001,Aral Sea,2922224.0,None


In [29]:
# print the "Country" values correspond to a None "Country Code"
df[df["Country Code"].isnull()]["Country"].unique()

array(['Africa', 'Aral Sea', 'Central America', 'Central Asia',
       'Eastern Africa', 'Kosovo', 'OECS Countries',
       'Red Sea and Gulf of Aden', 'West Bank and Gaza', 'Western Africa',
       'Western Balkans', 'Mekong', 'Caribbean', 'Latin America',
       'Southern Africa', 'Europe and Central Asia',
       'East Asia and Pacific', 'Caucasus', 'World', 'Andean Countries',
       'Central Africa', 'Middle East and North Africa',
       'Pacific Islands', 'Asia', 'South Asia',
       'Eastern and Southern Africa', 'St Maarten',
       'Western and Central Africa', 'Multi-Regional', 'South East Asia'],
      dtype=object)

#### Workbench

In [160]:
csv_gdp = "gdp_countries_data.csv"
df_gdp = pd.read_csv(csv_gdp)

In [161]:
df_gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.962907e+09,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.558906e+09,3.103184e+09,3.544708e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.992556e+11,8.297383e+11,9.399593e+11,1.012521e+12,1.006191e+12,9.288802e+11,1.086531e+12,1.185138e+12,NaN,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426650e+10,NaN,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.041165e+10,1.113592e+10,1.195171e+10,1.268581e+10,1.384900e+10,1.487476e+10,...,7.692632e+11,6.921149e+11,6.856303e+11,7.681582e+11,8.234056e+11,7.869624e+11,8.449275e+11,8.753937e+11,NaN,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.049642e+10,5.276162e+10,7.369016e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.067828e+11,NaN,NaN


In [ ]:
# Assuming `df` is the original dataframe
# Drop columns that are not needed
df_gdp = df_gdp.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 68'])

# Melt the dataframe
df_melted = df_gdp.melt(id_vars=['Country Name'], var_name='Year', value_name='GDP')

# Rename the columns
df_melted.columns = ['Country', 'Year', 'GDP']

df_melted.head()

In [154]:
# print the column datatypes of df and df_melted respectively
print(df.dtypes)
print(df_melted.dtypes)

Year              int64
Country          object
Award Amount    float64
dtype: object
Country     object
Year        object
GDP        float64
dtype: object


In [155]:
# convert the "Year" column in df to string
df['Year'] = df['Year'].astype(str)

In [156]:
# take df as the main dataframe and add the GDP values to it where year and country values match
df = df.merge(df_melted, on=["Country", "Year"], how="left")

In [157]:
df.head()

,Year,Country,Award Amount,GDP
0,2001,Africa,2259197.0,NaN
1,2001,Albania,31015512.0,3.922101e+09
2,2001,Algeria,53339668.0,5.474470e+10
3,2001,Angola,3976897.0,8.936079e+09
4,2001,Aral Sea,2922224.0,NaN


(3316, 4)

In [159]:
df["Year"].unique()

array(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype=object)

##### Workbench 2

In [3]:
# import relevant libraries
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pycountry

***

We change two char country codes in df_new into three char codes

In [80]:
csv_new = "Contract_Awards_in_Investment_Project_Financing.csv"
df_new = pd.read_csv(csv_new)

df_new.head()

,As of Date,Fiscal Year,Region,Borrower Country,Borrower Country Code,Project ID,Project Name,Project Global Practice,Procurement Category,Procurement Method,WB Contract Number,Contract Description,Borrower Contract Reference Number,Contract Signing Date,Supplier ID,Supplier,Supplier Country,Supplier Country Code,Supplier Contract Amount (USD),Review type
0,"Jun 21, 2024",2024,LATIN AMERICA AND CARIBBEAN,Colombia,CO,P162594,Multipurpose Cadaster Project,Information and Communications Technologies;Ag...,Consultant Services,Individual Consultant Selection,1783730,CONTRATAR LOS SERVICIOS DE CONSULTOR?A PARA LA...,CO-SNR-413278-CS-INDV,"May 16, 2024",850938.0,ELIANA MILENA BONILLA,Colombia,CO,32123.28,Post
1,"Jun 21, 2024",2024,Eastern and Southern Africa,Eastern and Southern Africa,NaN,P176517,"De-risking, inclusion and value enhancement of...","Financial Sector;Agriculture, Fishing and Fore...",Consultant Services,Quality And Cost-Based Selection,1783729,Recrutement d?un Cabinet pour la r?alisation d...,18',"Mar 21, 2024",796923.0,CABINET QSOLUTIONS,Djibouti,DJ,77214.66,Post
2,"Jun 21, 2024",2024,Eastern and Southern Africa,Angola,AO,P177004,Climate Resilience and Water Security in Angol...,"Water, Sanitation and Waste Management;Agricul...",Consultant Services,Individual Consultant Selection,1783728,5CS3/UCP/R/22? HYDROLOGIST / HYDROGEOLOGIST,5CS3/UCP/R/22,"Jul 14, 2023",402694.0,MIGUEL ANGEL ALONSO,Spain,ES,371600.00,Post
3,"Jun 21, 2024",2024,Eastern and Southern Africa,Eastern and Southern Africa,NaN,P176517,"De-risking, inclusion and value enhancement of...","Financial Sector;Agriculture, Fishing and Fore...",Consultant Services,Quality And Cost-Based Selection,1783727,Recrutement d?un cabinet (juriste + v?t?rinair...,12,"Mar 21, 2024",795668.0,GROUPEMEMENT GHALEB & ARREH-DEIL,Djibouti,DJ,130020.64,Post
4,"Jun 21, 2024",2024,LATIN AMERICA AND CARIBBEAN,Haiti,HT,P168951,Cap Haitien Urban Development Project,Transportation;Public Administration,Consultant Services,Direct Selection,1783726,POINT FOCAL ? LA MAIRIE DU CAP-HA?TIEN,INDV-001-CHUD/2023,"Nov 01, 2023",870107.0,NELSON DESHOMMES,Haiti,HT,15000.00,Post


In [81]:
# replace name of the column
df_new.rename(columns={"Borrower Country Code": "Country Code"}, inplace=True)

In [82]:
# replace two char country codes with three char country codes
def get_alpha_3(alpha_2_code):
    try:
        # get the two char code and return three char code
        return pycountry.countries.get(alpha_2=alpha_2_code).alpha_3
    except:
        # if the country code is not available return nan
        return np.nan

In [83]:
df_new["Country Code"] = df_new["Country Code"].apply(get_alpha_3)

df_new.shape

(235490, 20)

In [84]:
# drop rows with nan values in the "Country Code" column
df_new.dropna(subset=["Country Code"], inplace=True)
df_new.shape

(211848, 20)

In [85]:
df_new.head()

,As of Date,Fiscal Year,Region,Borrower Country,Country Code,Project ID,Project Name,Project Global Practice,Procurement Category,Procurement Method,WB Contract Number,Contract Description,Borrower Contract Reference Number,Contract Signing Date,Supplier ID,Supplier,Supplier Country,Supplier Country Code,Supplier Contract Amount (USD),Review type
0,"Jun 21, 2024",2024,LATIN AMERICA AND CARIBBEAN,Colombia,COL,P162594,Multipurpose Cadaster Project,Information and Communications Technologies;Ag...,Consultant Services,Individual Consultant Selection,1783730,CONTRATAR LOS SERVICIOS DE CONSULTOR?A PARA LA...,CO-SNR-413278-CS-INDV,"May 16, 2024",850938.0,ELIANA MILENA BONILLA,Colombia,CO,32123.28,Post
2,"Jun 21, 2024",2024,Eastern and Southern Africa,Angola,AGO,P177004,Climate Resilience and Water Security in Angol...,"Water, Sanitation and Waste Management;Agricul...",Consultant Services,Individual Consultant Selection,1783728,5CS3/UCP/R/22? HYDROLOGIST / HYDROGEOLOGIST,5CS3/UCP/R/22,"Jul 14, 2023",402694.0,MIGUEL ANGEL ALONSO,Spain,ES,371600.00,Post
4,"Jun 21, 2024",2024,LATIN AMERICA AND CARIBBEAN,Haiti,HTI,P168951,Cap Haitien Urban Development Project,Transportation;Public Administration,Consultant Services,Direct Selection,1783726,POINT FOCAL ? LA MAIRIE DU CAP-HA?TIEN,INDV-001-CHUD/2023,"Nov 01, 2023",870107.0,NELSON DESHOMMES,Haiti,HT,15000.00,Post
5,"Jun 21, 2024",2024,Eastern and Southern Africa,Sao Tome and Principe,STP,P180982,Coastal Resilience and Sustainable Tourism Pro...,NaN,Consultant Services,Quality And Cost-Based Selection,1783725,Designs for the mitigation of the erosion prob...,07/PPA/CARSTP/C/2023,"May 30, 2024",822315.0,CDR INTERNATIONAL B.V.,Netherlands,NL,527374.50,Post
6,"Jun 21, 2024",2024,Eastern and Southern Africa,Tanzania,TZA,P164920,Tanzania Roads to Inclusion and Socioeconomic ...,Transportation,Civil Works,Request for Bids,1783724,Construction of 3 stone masonry arch culverts ...,TZ-TARURA-413723-CW-RFB,"May 17, 2024",856254.0,OTONDE CONSTRUCTION & GENERAL SUPPLIES LIMITED,Tanzania,TZ,102139.72,Post


***

We work on old dataset to provide each country with a country code

In [100]:
csv_old = "Contract_Awards_in_Investment_Project_Financing__since_FY_2001_-_FY_2016_.csv"
df_old = pd.read_csv(csv_old)

df_old.head()

,As of Date,Fiscal Year,Region,Borrower Country,Project ID,Project Name,Project Global Practice,Procurement Category,Procurement Method,Contract Number,Contract Description,Borrower Contract Reference Number,Contract Signing Date,Supplier,Supplier Country,Supplier Contract Amount
0,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541010,Project Coordinator,2016-1,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,17535
1,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541011,Clinical Specialist,2016-2,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,34143
2,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541023,Information and Technology Specialist,2016-3,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,15591
3,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541030,Implementation and Procurement Specialist,2016-4,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,13045
4,03/11/2022 12:00:00 AM,2016,MIDDLE EAST AND NORTH AFRICA,Lebanon,P118187,Second Education Development Project,Education,Consultant Services,Direct Selection,1538291,Project Assistant,S34-IC-034,02/01/2016 12:00:00 AM,NIHAL NABAA,Lebanon,21600


In [101]:
df_old.shape

(146802, 16)

In [102]:
# get country code of each country in the old dataset concerning the "Borrower Country" column
df_old["Country Code"] = df_old["Borrower Country"].apply(
    lambda x: pycountry.countries.get(name=x).alpha_3 if pycountry.countries.get(name=x) else np.nan
    )

In [99]:
# remove rows with nan values in the "Country Code" column
df_old.dropna(subset=["Country Code"], inplace=True)

In [103]:
df_old.shape

(146802, 17)

In [104]:
df_old.head()

,As of Date,Fiscal Year,Region,Borrower Country,Project ID,Project Name,Project Global Practice,Procurement Category,Procurement Method,Contract Number,Contract Description,Borrower Contract Reference Number,Contract Signing Date,Supplier,Supplier Country,Supplier Contract Amount,Country Code
0,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541010,Project Coordinator,2016-1,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,17535,MNG
1,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541011,Clinical Specialist,2016-2,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,34143,MNG
2,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541023,Information and Technology Specialist,2016-3,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,15591,MNG
3,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541030,Implementation and Procurement Specialist,2016-4,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,13045,MNG
4,03/11/2022 12:00:00 AM,2016,MIDDLE EAST AND NORTH AFRICA,Lebanon,P118187,Second Education Development Project,Education,Consultant Services,Direct Selection,1538291,Project Assistant,S34-IC-034,02/01/2016 12:00:00 AM,NIHAL NABAA,Lebanon,21600,LBN


***

Now we combine two datasets for their country codes

In [105]:
# group df_new by "Borrower Country" and "Country Code" and sum the "Supplier Contract Amount (USD)" column for each year
df_new = df_new.groupby(["Borrower Country", "Country Code", "Fiscal Year"])["Supplier Contract Amount (USD)"].sum().reset_index()
df_new.head()

,Borrower Country,Country Code,Fiscal Year,Supplier Contract Amount (USD)
0,Afghanistan,AFG,2017,1.879657e+08
1,Afghanistan,AFG,2018,1.408374e+08
2,Afghanistan,AFG,2019,8.300205e+08
3,Afghanistan,AFG,2020,2.090723e+08
4,Afghanistan,AFG,2021,1.628221e+08


In [117]:
# print the country name and corresponding country codes
df_new[df_new["Borrower Country"] == "Turkiye"]

,Borrower Country,Country Code,Fiscal Year,Supplier Contract Amount (USD)
876,Turkiye,TUR,2017,7.828628e+07
877,Turkiye,TUR,2018,3.960666e+07
878,Turkiye,TUR,2019,1.238145e+09
879,Turkiye,TUR,2020,1.939322e+08
880,Turkiye,TUR,2021,1.276939e+09
881,Turkiye,TUR,2022,1.332820e+08
882,Turkiye,TUR,2023,3.062545e+08
883,Turkiye,TUR,2024,6.692237e+08


***

In [121]:
csv_gdp = "gdp_countries_data.csv"
df_gdp = pd.read_csv(csv_gdp)

df_gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.962907e+09,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.558906e+09,3.103184e+09,3.544708e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.992556e+11,8.297383e+11,9.399593e+11,1.012521e+12,1.006191e+12,9.288802e+11,1.086531e+12,1.185138e+12,NaN,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426650e+10,NaN,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.041165e+10,1.113592e+10,1.195171e+10,1.268581e+10,1.384900e+10,1.487476e+10,...,7.692632e+11,6.921149e+11,6.856303e+11,7.681582e+11,8.234056e+11,7.869624e+11,8.449275e+11,8.753937e+11,NaN,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.049642e+10,5.276162e+10,7.369016e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.067828e+11,NaN,NaN


***

In [73]:
#
# IMPORT THE DATA
# 

csv_new = "Contract_Awards_in_Investment_Project_Financing.csv"
csv_old = "Contract_Awards_in_Investment_Project_Financing__since_FY_2001_-_FY_2016_.csv"

df_new = pd.read_csv(csv_new)
df_old = pd.read_csv(csv_old)

#
# CLEAN THE DATA
#

# make the region names consistent in both datasets by making them all lowercase except the first letters
df_new["Region"] = df_new["Region"].str.title()
df_old["Region"] = df_old["Region"].str.title()

# change the region names to match the new dataset
df_old["Region"] = df_old["Region"].replace("Africa East", "Eastern And Southern Africa")
df_old["Region"] = df_old["Region"].replace("Africa West", "Western And Central Africa")

# change the country names of "Eastern Africa" and "Western Africa" to match the new dataset
df_old.loc[df_old["Region"] == "Eastern And Southern Africa", "Borrower Country"] = df_old.loc[
    df_old["Region"] == "Eastern And Southern Africa", "Borrower Country"
    ].replace(
        "Eastern Africa", "Eastern And Southern Africa"
        )
df_old.loc[df_old["Region"] == "Western And Central Africa", "Borrower Country"] = df_old.loc[
    df_old["Region"] == "Western And Central Africa", "Borrower Country"
    ].replace(
        "Western Africa", "Western And Central Africa"
        ) 

# change "Turkey" to "Turkiye" in the old dataset
df_old.loc[df_old["Borrower Country"] == "Turkey", "Borrower Country"] = "Turkiye"

#
# MERGE THE DATAFRAMES
# 

# merge the dataframes with the relevant categories as following:
# Fiscal Year, Region, Borrower Country, Project Global Practice, Supplier Contract Amount (USD)
df_new = df_new[['Fiscal Year',
                 'Region',
                 'Borrower Country',
                 'Project Global Practice',
                 'Supplier Contract Amount (USD)']]
df_old = df_old[['Fiscal Year',
                 'Region',
                 'Borrower Country',
                 'Project Global Practice',
                 'Supplier Contract Amount']]

# rename the columns to match the old dataframe
df_new.rename(columns={'Supplier Contract Amount (USD)': 'Supplier Contract Amount'}, inplace=True)

# merge the dataframes, removing any duplicates
df = pd.concat([df_new, df_old], ignore_index=True)
df.drop_duplicates(inplace=True)

# store the unique values of Project Global Practice
project_global_practices = df["Project Global Practice"].unique()

df.head()

,Fiscal Year,Region,Borrower Country,Project Global Practice,Supplier Contract Amount
0,2024,Latin America And Caribbean,Colombia,Information and Communications Technologies;Ag...,32123.28
1,2024,Eastern And Southern Africa,Eastern and Southern Africa,"Financial Sector;Agriculture, Fishing and Fore...",77214.66
2,2024,Eastern And Southern Africa,Angola,"Water, Sanitation and Waste Management;Agricul...",371600.00
3,2024,Eastern And Southern Africa,Eastern and Southern Africa,"Financial Sector;Agriculture, Fishing and Fore...",130020.64
4,2024,Latin America And Caribbean,Haiti,Transportation;Public Administration,15000.00


In [67]:
# combine the Supplier Contract Amount columns for a given country and year dropping Project Global Practice column
df = df.groupby(['Fiscal Year', 'Region', 'Borrower Country']).sum().reset_index()

df.head()

C:\Users\husme\AppData\Local\Temp\ipykernel_4348\2421625099.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['Fiscal Year', 'Region', 'Borrower Country']).sum().reset_index()


,Fiscal Year,Region,Borrower Country,Supplier Contract Amount
0,2001,Africa,Africa,2.259197e+06
1,2001,East Asia And Pacific,Cambodia,2.185357e+07
2,2001,East Asia And Pacific,China,1.204774e+09
3,2001,East Asia And Pacific,Indonesia,1.100204e+08
4,2001,East Asia And Pacific,"Korea, Republic of",5.506480e+05


In [68]:
df.rename(columns={"Borrower Country": "Country", "Fiscal Year": "Year", "Supplier Contract Amount": "Award Amount"}, inplace=True)

In [69]:
df.head()

,Year,Region,Country,Award Amount
0,2001,Africa,Africa,2.259197e+06
1,2001,East Asia And Pacific,Cambodia,2.185357e+07
2,2001,East Asia And Pacific,China,1.204774e+09
3,2001,East Asia And Pacific,Indonesia,1.100204e+08
4,2001,East Asia And Pacific,"Korea, Republic of",5.506480e+05


In [70]:
df["Country Code"] = df["Country"].apply(lambda x: pycountry.countries.get(name=x).alpha_3 if pycountry.countries.get(name=x) else np.nan)

In [71]:
df.head()

,Year,Region,Country,Award Amount,Country Code
0,2001,Africa,Africa,2.259197e+06,NaN
1,2001,East Asia And Pacific,Cambodia,2.185357e+07,KHM
2,2001,East Asia And Pacific,China,1.204774e+09,CHN
3,2001,East Asia And Pacific,Indonesia,1.100204e+08,IDN
4,2001,East Asia And Pacific,"Korea, Republic of",5.506480e+05,KOR


In [72]:
# print the country values that are not available in the pycountry library
df[df["Country Code"].isnull()]["Country"].unique()

array(['Africa', 'Vietnam', 'Eastern And Southern Africa', 'Tanzania',
       'Aral Sea', 'Central Asia', 'Kosovo', 'Kyrgyz Republic', 'Moldova',
       'Slovak Republic', 'Turkiye', 'Western Balkans', 'Bolivia',
       'Central America', 'OECS Countries', 'St. Kitts and Nevis',
       'St. Lucia', 'Venezuela, Republica Bolivariana de',
       'Egypt, Arab Republic of', 'Red Sea and Gulf of Aden',
       'West Bank and Gaza', 'Yemen, Republic of', "Cote d'Ivoire",
       'Gambia, The', 'Western And Central Africa', 'Mekong',
       'Congo, Democratic Republic of', 'St. Vincent and the Grenadines',
       'Congo, Republic of', 'Southern Africa', 'Caribbean',
       'Latin America', 'Czech Republic', 'Europe and Central Asia',
       'East Asia and Pacific', 'Caucasus', 'World', 'Andean Countries',
       'Central Africa', 'Pacific Islands',
       'Middle East and North Africa', 'Asia', 'South Asia',
       'Eastern and Southern Africa', 'St Maarten',
       'Western and Central Africa'

In [118]:
csv_new = "Contract_Awards_in_Investment_Project_Financing.csv"
csv_old = "Contract_Awards_in_Investment_Project_Financing__since_FY_2001_-_FY_2016_.csv"

df_new = pd.read_csv(csv_new)
df_old = pd.read_csv(csv_old)


In [119]:
df_new.head()

,As of Date,Fiscal Year,Region,Borrower Country,Borrower Country Code,Project ID,Project Name,Project Global Practice,Procurement Category,Procurement Method,WB Contract Number,Contract Description,Borrower Contract Reference Number,Contract Signing Date,Supplier ID,Supplier,Supplier Country,Supplier Country Code,Supplier Contract Amount (USD),Review type
0,"Jun 21, 2024",2024,LATIN AMERICA AND CARIBBEAN,Colombia,CO,P162594,Multipurpose Cadaster Project,Information and Communications Technologies;Ag...,Consultant Services,Individual Consultant Selection,1783730,CONTRATAR LOS SERVICIOS DE CONSULTOR?A PARA LA...,CO-SNR-413278-CS-INDV,"May 16, 2024",850938.0,ELIANA MILENA BONILLA,Colombia,CO,32123.28,Post
1,"Jun 21, 2024",2024,Eastern and Southern Africa,Eastern and Southern Africa,NaN,P176517,"De-risking, inclusion and value enhancement of...","Financial Sector;Agriculture, Fishing and Fore...",Consultant Services,Quality And Cost-Based Selection,1783729,Recrutement d?un Cabinet pour la r?alisation d...,18',"Mar 21, 2024",796923.0,CABINET QSOLUTIONS,Djibouti,DJ,77214.66,Post
2,"Jun 21, 2024",2024,Eastern and Southern Africa,Angola,AO,P177004,Climate Resilience and Water Security in Angol...,"Water, Sanitation and Waste Management;Agricul...",Consultant Services,Individual Consultant Selection,1783728,5CS3/UCP/R/22? HYDROLOGIST / HYDROGEOLOGIST,5CS3/UCP/R/22,"Jul 14, 2023",402694.0,MIGUEL ANGEL ALONSO,Spain,ES,371600.00,Post
3,"Jun 21, 2024",2024,Eastern and Southern Africa,Eastern and Southern Africa,NaN,P176517,"De-risking, inclusion and value enhancement of...","Financial Sector;Agriculture, Fishing and Fore...",Consultant Services,Quality And Cost-Based Selection,1783727,Recrutement d?un cabinet (juriste + v?t?rinair...,12,"Mar 21, 2024",795668.0,GROUPEMEMENT GHALEB & ARREH-DEIL,Djibouti,DJ,130020.64,Post
4,"Jun 21, 2024",2024,LATIN AMERICA AND CARIBBEAN,Haiti,HT,P168951,Cap Haitien Urban Development Project,Transportation;Public Administration,Consultant Services,Direct Selection,1783726,POINT FOCAL ? LA MAIRIE DU CAP-HA?TIEN,INDV-001-CHUD/2023,"Nov 01, 2023",870107.0,NELSON DESHOMMES,Haiti,HT,15000.00,Post


In [120]:
df_old.head()

,As of Date,Fiscal Year,Region,Borrower Country,Project ID,Project Name,Project Global Practice,Procurement Category,Procurement Method,Contract Number,Contract Description,Borrower Contract Reference Number,Contract Signing Date,Supplier,Supplier Country,Supplier Contract Amount
0,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541010,Project Coordinator,2016-1,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,17535
1,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541011,Clinical Specialist,2016-2,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,34143
2,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541023,Information and Technology Specialist,2016-3,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,15591
3,03/11/2022 12:00:00 AM,2016,EAST ASIA AND PACIFIC,Mongolia,P131290,E-Health Project,Health,Consultant Services,Individual Consultant Selection,1541030,Implementation and Procurement Specialist,2016-4,06/07/2016 12:00:00 AM,INDIVIDUAL CONSULTANT,World,13045
4,03/11/2022 12:00:00 AM,2016,MIDDLE EAST AND NORTH AFRICA,Lebanon,P118187,Second Education Development Project,Education,Consultant Services,Direct Selection,1538291,Project Assistant,S34-IC-034,02/01/2016 12:00:00 AM,NIHAL NABAA,Lebanon,21600
